In [2]:
import folium
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)
pd.set_option('display.max_columns', None)

In [3]:
from folium.plugins import MarkerCluster

Order of Events: 

- Import intermediate data
- Change data type to datetime for relevant columns - SIKE
- Filter kick_data on "live", "successful", and "failed".
- Merge tables on kick_data ['city', state_loc, country_loc]


In [4]:
kick_data = pd.read_csv('../../data/02_intermediate/kick_inter.csv')
world = pd.read_csv('../../data/01_raw/worldcities.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11,15,16,17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
UScities = pd.read_csv('../../data/01_raw/uscitiesv1.5.csv')

From "world" - I want to keep 'lat', 'lng', 'city_ascii', 'country', 'iso2'

In [6]:
world.drop(columns = ['iso3', 'admin_name', 'capital', 'population', 'id', 'city'], inplace=True)

In [23]:
kick_data.head()
world.loc[(world.iso2 == 'US')]
UScities.drop(columns = ['county_name', 'zips', 'population', 'population_proper', 
                         'id', 'city', 'density', 'source', 'timezone', 'incorporated', 'zips', 'county_fips'], inplace=True)

In [28]:
world.columns

Index(['city_ascii', 'lat', 'lng', 'country', 'iso2'], dtype='object')

In [8]:
drop = ['blurb', 'converted_pledged_amount',
       'created_at', 'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'is_backing', 'is_starrable', 'is_starred',
       'last_update_published_at', 'launched_at', 'permissions',
       'pledged', 'slug', 'source_url', 'spotlight', 'staff_pick',
       'state_changed_at', 'static_usd_rate', 'unread_messages_count',
       'unseen_activity_count', 'usd_pledged', 'usd_type',
       'creator_name', 'creator_slug', 'country']

In [10]:
kick_data.drop(columns = drop, inplace = True)

In [11]:
plot_filter = ((kick_data.state == 'successful') | (kick_data.state == 'failed') | (kick_data.state == 'live'))
kick_to_plot = kick_data.loc[plot_filter]

Kick_to_plot has 314399 rows. Of which, 124793 do not have a value for city. That leaves 189606 to potentially be renderded on the map.

In [27]:
kick_to_map1 = kick_to_plot.merge(UScities[['city_ascii', 'state_id', 'lat', 'lng']], how='left', left_on=['city', 'state_loc'], right_on=['city_ascii', 'state_id'])


In [30]:
kick_to_map1.head()

,backers_count,goal,id,name,state,urls,sub_category,overall_category,city,country_loc,state_loc,city_ascii,state_id,lat,lng
0,1,1500.0,654869661,Josiah Gennell: Creating First EP,live,"{""web"":{""project"":""https://www.kickstarter.com...",Indie Rock,music/indie rock,Methuen,US,MA,Methuen,MA,42.7340,-71.1889
1,0,5000.0,1290757180,Giuliano Clothing: Modern Fashion,live,"{""web"":{""project"":""https://www.kickstarter.com...",Fashion,fashion,Toronto,CA,ON,NaN,NaN,NaN,NaN
2,1,4000.0,1747360837,CincyPet Magazine,live,"{""web"":{""project"":""https://www.kickstarter.com...",Periodicals,publishing/periodicals,Cincinnati,US,OH,Cincinnati,OH,39.1412,-84.5059
3,4,370.0,592255111,Bad-ass Women Enamel Pins,live,"{""web"":{""project"":""https://www.kickstarter.com...",Art,art,Dallas,US,TX,Dallas,TX,32.7937,-96.7662
4,1,10000.0,1561837546,Creative Craft Kits,live,"{""web"":{""project"":""https://www.kickstarter.com...",Crafts,crafts,La Mesa,US,CA,La Mesa,CA,32.7703,-117.0204


It worked! No extra rows added this time! kick_to_map1.shape = (314399, 15)
We also have 190495 null values in Lat/Lng. Let's see if we can get that down.

Merging magic got my number of lat/lng nulls down to 145738. Exceptable to move forward with mapping. Going to remove anything else that's null at this point. 

In [34]:
kick_to_map2 = kick_to_map1.merge(world[['city_ascii', 'lat', 'lng','iso2']],  how="left", left_on=['city','country_loc'], right_on=['city_ascii', 'iso2'])

In [45]:
kick_to_map2.columns


Index(['backers_count', 'goal', 'id', 'name', 'state', 'urls', 'sub_category',
       'overall_category', 'city', 'country_loc', 'state_loc', 'city_ascii_x',
       'state_id', 'lat_x', 'lng_x', 'city_ascii_y', 'lat_y', 'lng_y', 'iso2',
       'lat', 'lng'],
      dtype='object')

In [40]:
kick_to_map2.lat_y.isna().sum()

151642

In [41]:
kick_to_map2['lat'] = kick_to_map2['lat_x'].fillna(kick_to_map2['lat_y'])
kick_to_map2['lng'] = kick_to_map2['lng_x'].fillna(kick_to_map2['lng_y'])

In [46]:
kick_to_map2.drop(columns=['city_ascii_x','state_id', 
                           'lat_x', 'lng_x', 'city_ascii_y', 
                           'lat_y', 'lng_y', 'iso2'], inplace = True)

In [51]:
kick_to_map2.dropna(subset=['lat', 'lng'], inplace=True)

In [54]:
kick_to_map2.shape

(275048, 13)

In [55]:
kick_to_map2.to_pickle('../../data/02_intermediate/kick_World_map.pkl')